In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uttarakhand-tweets/Uttarakhand.csv


In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
import spacy
nlp =spacy.load('en_core_web_sm')


df = pd.read_csv("/kaggle/input/uttarakhand-tweets/Uttarakhand.csv")


In [3]:
questions = [
    'What is the Emergency helpline number?',
    'What happened in Uttarakhand?',
    'How many people are killed?',
    'Which tunnel completely blocked due to debris?'
]

In [4]:
df

,tweet
0,Horrible news out of #Uttarakhand
1,Prayer for #Uttarakhand 🙏🙏
2,Ohhh God Please Save #Uttarakhand
3,Extremely Sad To Hear About The #Uttarakhand G...
4,Prayers for #Uttarakhand 🙏🙏
...,...
8141,"😊\n\nHope everyone is fine, Prayers 🙏\n#Uttara..."
8142,What a responsible PM would do in this Situati...
8143,Scary scenes coming from #Uttarakhand \n\nPray...
8144,ITBP jawans in search and rescue operation in ...


In [5]:
df = df.drop_duplicates().reset_index(drop = True)
df

,tweet
0,Horrible news out of #Uttarakhand
1,Prayer for #Uttarakhand 🙏🙏
2,Ohhh God Please Save #Uttarakhand
3,Extremely Sad To Hear About The #Uttarakhand G...
4,Prayers for #Uttarakhand 🙏🙏
...,...
7494,May the God force be with you #Uttarakhand
7495,"😊\n\nHope everyone is fine, Prayers 🙏\n#Uttara..."
7496,What a responsible PM would do in this Situati...
7497,ITBP jawans in search and rescue operation in ...


In [6]:
df['tweet'][0]

'Horrible news out of #Uttarakhand'

In [7]:
context = ''
for i in range(8):
    for text in df['tweet'][i]:
        context += text
context

'Horrible news out of #UttarakhandPrayer for #Uttarakhand 🙏🙏Ohhh God Please Save #UttarakhandExtremely Sad To Hear About The #Uttarakhand Glacier Break, Praying For The Safety Of Missing People... 🙏\n\nHelpline… https://t.co/a8HXQ6Y3HbPrayers for #Uttarakhand 🙏🙏Prayer for #Uttarakhand.\n🙏🙏ALERT 🚨 #Uttarakhand \n\nEmergency 🆘 numbers 1070  9557444486 \n\nPlease follow all official advisories.\n\n#Uttarakhand #ChamoliHeavy Devastation Due To Glacier breakage in Chamoli, Uttrakhand.Very sad news...Geographical issues of our country… https://t.co/hFmkb6D9bX'

In [8]:
context1 = ''
for i in range(75,85):
    for text in df['tweet'][i]:
        context1 += text
context1

"ITBP personnel digging to open Tapovan Tunnel which is completely blocked due to debris. Many reported to be trappe… https://t.co/Ta3b2cDY9e#UttarakhandDisaster #Uttarakhand praying 🙏 for everyone's safety.Thoughts &amp; Prayers for Uttarakhand!\n#UttarakhandALERT 🚨 #Uttarakhand \n\nEmergency 🆘 numbers 1070  9557444486 \n\nPlease follow all official advisories.\n\n#UttarakhandPraying for safety of all #UttarakhandAbout 100 to 150 people are feared to be killed in the #flashflood in #Uttarakhand #Chamoli district, news agency A… https://t.co/nScJsRSK5FFlash floods in #Uttarakhand:\nFour Army columns, Two Medical teams,  one Engineering Task Force deployed at #Ringi… https://t.co/Gp6ZnIhU4BDistressed by the tragic disaster that unfolded in Uttarakhand. Praying for everyone’s safety. Hope those stranded… https://t.co/LL8Jz1nO0d#Uttarakhand Distressing to hear about the glacier breaking off in #Uttarakhand Praying everyone’s safety there 🙏we all are with u #Uttarakhand"

In [9]:
import re
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
final_context = deEmojify(context)

In [10]:
final_context

'Horrible news out of #UttarakhandPrayer for #Uttarakhand Ohhh God Please Save #UttarakhandExtremely Sad To Hear About The #Uttarakhand Glacier Break, Praying For The Safety Of Missing People... \n\nHelpline… https://t.co/a8HXQ6Y3HbPrayers for #Uttarakhand Prayer for #Uttarakhand.\nALERT  #Uttarakhand \n\nEmergency 🆘 numbers 1070  9557444486 \n\nPlease follow all official advisories.\n\n#Uttarakhand #ChamoliHeavy Devastation Due To Glacier breakage in Chamoli, Uttrakhand.Very sad news...Geographical issues of our country… https://t.co/hFmkb6D9bX'

In [11]:
from transformers import BertTokenizer, TFBertForQuestionAnswering
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [12]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [13]:
def answer_question(question, answer_text):

    input_ids = tokenizer.encode(question, answer_text)
    print('Query has {:,} tokens.\n'.format(len(input_ids)))
    sep_index = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)

    outputs = model(torch.tensor([input_ids]), 
                    token_type_ids=torch.tensor([segment_ids]),
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits


    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)


    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    answer = tokens[answer_start]


    for i in range(answer_start + 1, answer_end + 1):
        

        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        

        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

In [14]:
print(questions[0])
print(answer_question(questions[0], final_context))

What is the Emergency helpline number?
Query has 174 tokens.

Answer: "1070 9557444486"
None


In [15]:
print(questions[1])
print(answer_question(questions[1], final_context))

What happened in Uttarakhand?
Query has 173 tokens.

Answer: "glacier break"
None


In [16]:
print(questions[2])
print(answer_question(questions[2], context1))

How many people are killed?
Query has 283 tokens.

Answer: "100 to 150"
None


In [17]:
print(questions[3])
print(answer_question(questions[3], context1))

Which tunnel completely blocked due to debris?
Query has 285 tokens.

Answer: "tapovan tunnel"
None
